# 💫 [Day 25](https://adventofcode.com/2019/day/25)

In [1]:
from collections import defaultdict

def run_program(p, inputs, init_op=0, init_base=0):
    """Intcode as usual"""
    # Note: p can be either a list or a dictionnary (for heavy memory usage)
    #Inputs are given in reverse order (pop)
    op = init_op
    relative_base = init_base
    last_diagnostic = []
    while p[op] != 99:
        codes = "%05d" % p[op]
        codes = [int(codes[0]), int(codes[1]), int(codes[2]), int(codes[3:])]
        # inputs
        if codes[-1] == 3:
            if not len(inputs):
                # Game is waiting for next move
                break
            assert codes[1] == 0
            p[p[op + 1] + (relative_base if codes[2] == 2 else 0)] = inputs.pop()
            op += 2
        # unary ops
        elif codes[-1] in  [4, 9]:
            # read parameter
            assert codes[1] == 0
            param = p[op + 1]
            if (codes[2] % 2) == 0:
                try:
                    param = p[param + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    param = 0
            # output
            if codes[-1] == 4:
                last_diagnostic.append(param)
            # update relative base
            else:
                relative_base += param
            # next instr
            op += 2
        else:
            # read parameters in correct mode
            x, y = p[op + 1], p[op + 2]
            if (codes[2] % 2) == 0:
                try:
                    x = p[x + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    x = 0
            if not (codes[1] % 2):
                try:
                    y = p[y + (relative_base if codes[1] == 2 else 0)]
                except IndexError:
                    y = 0
            # Read target and allocate more memory if needed
            target = p[op + 3] + (relative_base if codes[0] == 2 else 0)
            #if target >= len(p): 
            #    p += [0] * (target - len(p) + 1)
            # addition and multiplication
            if codes[-1] in [1, 2]:
                p[target] = x + y if codes[-1] == 1 else x * y
                op += 4
            # Comparison result
            elif codes[-1] == 7:
                p[target] = int(x < y)
                op += 4
            elif codes[-1] == 8:
                p[target] = int(x == y)
                op += 4
            # Jump if eq
            elif (codes[-1] == 5 and x != 0) or (codes[-1] == 6 and x == 0):
                op = y  
            # Jump instruction that failed their test
            else:
                op += 3
    return last_diagnostic, 0, op, relative_base


# For the game
import sys
import itertools

# For display
from io import BytesIO
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import base64
import numpy as np
from matplotlib import colors


def check_items(p, op, base, inventory):
    def rec_check(current_objs, inv, result):
        """Check weights for all items combination"""
        nonlocal p, op, base

        for i, obj in enumerate(inv):
            # take the object and go to checkpoint
            # NOTE: Assumes checkpoint is on the east
            inputs = "take {}\neast\n".format(obj)
            inputs = list(map(ord, inputs))
            output, _, op, base = run_program(
                p, inputs[::-1], init_op=op, init_base=base)
            output = ''.join(list(map(chr, output))).replace('\n', '<br>')
            
            # Store in result if test failed
            if "Alert" in output:
                res = "[<b>{}</b>]".format(', '.join(current_objs + [obj]))
                # If we're still too light, continue with this state
                if "heavier" in output:
                    res += " -> too light"
                    result.append(res)
                    solution = rec_check(current_objs + [obj], inv[i + 1:], result)
                    if solution is not None:
                        return solution
                # If we're too heavy, drop this object and continue from other state
                elif "lighter" in output:
                    res += " -> too heavy"
                    result.append(res)
                # In all cases, we drop the object back to revert to initial state
                inputs = "drop {}\n".format(obj)
                inputs = list(map(ord, inputs))
                _, _, op, base = run_program(p, inputs[::-1], init_op=op, init_base=base)
            # Otherwise, we found the solution
            else:
                return current_objs + [obj], output
        return None
    result = []
    solution = rec_check([], inventory, result)
    return result, solution
            

def play(program, iwidth='200px', n=21, path=[]):
    # Monitor current program state
    inputs = []
    op, base = 0, 0
    p = defaultdict(lambda: 0)
    for i, v in enumerate(program):
        p[i] = v
    # 0: ?, 1: dead end, 2: unexplored path, 3: visited path, 4: object, 5: current
    maze_map = np.zeros((n, n))
    i, j = n // 2, n // 2
    inv = []
    a = ''
    
    # For display
    text = widgets.HTML("Game start", layout=widgets.Layout(width='300px', height='400px'))
    image = widgets.HTML(layout=widgets.Layout(width='400px', height='400px'))
    inventory = widgets.HTML(layout=widgets.Layout(width='50px', height='400px'))
    display(widgets.HBox([text, image, inventory]))
    norm = colors.Normalize(vmin=0, vmax=5)
    cmap = 'tab20c_r'
    
    # Play the game
    while 1:
        # Execute last action
        output, _, op, base = run_program(p, inputs[::-1], init_op=op, init_base=base)
        s = ''.join(list(map(chr, output))).replace('\n', '<br>')
        # Game over
        if len(output) == 0:
            text.value = "<b>GAME OVER</b>"
            return 1
        
        # Update current estimate of map
        if not a.startswith('take'):
            maze_map[i, j] = 3
        if ("You can't go that way" not in s and 
            not "ejected back to the checkpoint" in s):
            if a == 'east':
                j += 1
            elif a == 'west':
                j -= 1
            elif a == 'north':
                i -= 1
            elif a == 'south':
                i += 1
        if "You don't " not in s:
            if a.startswith('take'):
                inv.append(a.split(' ', 1)[1])
                maze_map[i, j] = 3
            elif a.startswith('drop'):
                obj = a.split(' ', 1)[1]
                inv = [x for x in inv if x != obj]
                maze_map[i, j] = 4
        
        # Display current state and map estimate
        text.value = "<b>" + s + "</b>"
        inventory.value = ("<b>Inventory:</b><br>" + 
                           '<br>'.join("  - {}".format(x) for x in inv))
        maze_map[i, j] = 5
        if 'east' in s:
            maze_map[i, j + 1] = max(2, maze_map[i, j + 1])
        else:
            maze_map[i, j + 1] = max(1, maze_map[i, j + 1])
        if 'west' in s:
            maze_map[i, j - 1] = max(2, maze_map[i, j - 1])
        else:
            maze_map[i, j - 1] = max(1, maze_map[i, j - 1])
        if 'north' in s:
            maze_map[i - 1, j] = max(2, maze_map[i - 1, j])
        else:
            maze_map[i - 1, j] = max(1, maze_map[i - 1, j])
        if 'south' in s:
            maze_map[i + 1, j] = max(2, maze_map[i + 1, j])
        else:
            maze_map[i + 1, j] = max(1, maze_map[i + 1, j])
        if "Items here" in s:
            maze_map[i, j] = 4
        fig = plt.figure()
        bio = BytesIO()
        plt.imshow(maze_map, cmap=cmap, norm=norm)
        plt.axis('off')
        fig.savefig(bio, format='png')
        plt.close(fig)
        image.value = ("<b>Position ({}, {})</b>".format(i, j) + 
                       "<img src='data:image/png;base64," + 
                       base64.b64encode(bio.getvalue()).decode('UTF-8') + 
                       "'width='500px'/>".format(iwidth))
        
        # If reaching security checkpoint, test for items combinations automatically
        if "Security Checkpoint" in s:
            # Drop all items initially
            inputs = ''.join(["drop {}\n".format(i) for i in inv])
            inputs = list(map(ord, inputs))
            _, _, op, base = run_program(p, inputs[::-1], init_op=op, init_base=base)
            # Find best combination
            result, solution = check_items(p, op, base, inv)
            text.value = "<b>" + s + "</b>" + "<b>Checkpoint result:</b><br>" + "<br>".join(result)
            if solution is not None:
                #print("WTF", solution)
                #solution, output, p, op, base = solution
                text.value = ("<b>" + solution[1] + "</b>" + 
                              "<br><b>Items</b>:<br>" + '<br>'.join(solution[0]) +
                              "<br><br><b>Other trails:</b><br>" + "<br>".join(result))
                return
        
        # Ask for next command
        if len(path):
            a = path.pop(0)
        else:
            a = input()
        clear_output()
        display(widgets.HBox([text, image, inventory]))
        if a == 'e':
            a = 'east'
        elif a == 'n':
            a = 'north'
        elif a == 'w':
            a = 'west'
        elif a == 's':
            a = 'south'
        inputs = list(map(ord, a)) + [10]
            

In [2]:
with open('inputs/day25.txt', 'r') as f:
    inputs = list(map(int, f.read().split(',')))
# use path = [] for manual exploration instead
path = ['north', 'take festive hat', 'east', 'take prime number', 
        'west', 'west', 'take sand', 'east', 'south', 'east', 'north',
        'take weather machine', 'north', 'take mug', 'south', 'south',
        'east', 'north', 'east', 'east', 'take astronaut ice cream',
        'west', 'west', 'south', 'west', 'west', 'south', 'south',
        'take mutex', 'south', 'take boulder', 'east', 'south'
 ]

In [3]:
# Manual (or predetermined exploratoin) to find objects
# Automatic weight checking is triggered when arriving to the checkpoint
play(inputs, path=path)